In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth
import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables

#!jupyter-theme -t onedork

In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0#0.0005
scalarfield=True
vscalarfield=False


In [3]:
Tol=1.0
N=10
Nul=10
Nup=10**(80)#10**(20)#

scal=1
umax=13
vmax=1000


ru0=5.0
dr0v=.4

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    du00=1/N
    du0=du00
else:
    du00=M0/N
    du0=du00
dv00=du00
dv0=du0


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
rminus=sol.x[0]
rplus=sol.x[1]
rcosm=sol.x[2]

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989992
r-: 0.687750100079
rc: 1.31224990002
Number of points for highest iteration is 130000000,13000X10000
Total number of points is 130000000
Runtime: about 27.78 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])

rnpf=np.zeros((2,Nv))
signpf=np.zeros((2,Nv))
phinpf=np.zeros((2,Nv))
#dphinpuf=np.zeros((Nu),len(Elist))) 
massnpf=np.zeros((2,Nv))


#tempsignpu=np.zeros((Nu*max(Elist),Nv*max(Elist)))




rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,scalarfield)


print(len(rnpf))
print(rnpf)
#print(rnpf[1])
print()
#print(rnpf)

2
[[   5.      5.04    5.08 ...,  404.88  404.92  404.96]
 [   0.      0.      0.   ...,    0.      0.      0.  ]]



In [6]:
#drunp=np.empty((Nu))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=8.5
u2=9.5

###Applying Propagation Algorithm###

urange=[0.0]
drunp=np.array([0.0])
dsigunp=np.array([0.0])
dphinpuf=np.array([0.0])


drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0+Lambda*ru0**2/3)
if bdytype=="edd":
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
    dsigunp[0]=2*(3*Q**2-3*M0*r+r**4*Lambda)/(r*(-3*Q**2+r*(6*M0-3*r+r**3*Lambda)))
else:
    dsigunp[0]=0.0

i=0
dv=dv0
du0=M0/Nul
if bdytype=="edd":
    bdyvalue=1.0
else:
    bdyvalue=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

#print(rnp)
    
while max(urange)<umax and i<20000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    drunp=np.append(drunp,np.nan)
    dsigunp=np.append(dsigunp,np.nan)
    dphinpuf=np.append(dphinpuf,np.nan)
    
    
    du=abs(du0*Tol/mth.exp(np.nanmax(signpf[0,:])))
    
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(10*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    print("%.64f" % urange[i])
    print("%.80f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    signpf[1][0]=signpf[0][0]+du*dsigunp[i]
    dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    
    
    
    #print(rnpf[1][0])
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,Q,Lambda)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        
        #drunp[1][j+1]=answer[4]
        #dsigunp[1][j+1]=answer[5]
            
    #print(len(rnpf[1]))
    
    rnp.append(rnpf[1].reshape(1,Nv))
    signp.append(signpf[1].reshape(1,Nv))
    phinp.append(phinpf[1].reshape(1,Nv))
    massnp.append(massnpf[1].reshape(1,Nv))
    
    
    rnpf[0,:]=rnpf[1,:]
    phinpf[0,:]=phinpf[1,:]
    signpf[0,:]=signpf[1,:]
    massnpf[0,:]=massnpf[1,:]
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)

    urange.append(urange[i]+du) 
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    dumaxlist=[]
    i+=1
    
    #if i>1000:
        #break


0.0000000000000000000000000000000000000000000000000000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
0.0100000000000000002081668171172168513294309377670288085937500000
0.09995914756407302259599845228876802138984203338623046875000000000000000000000000
0.1099591475640730175999948414755635894834995269775390625000000000
0.09954760731370071447088321292540058493614196777343750000000000000000000000000000
0.2095067548777737320708780544009641744196414947509765625000000000
0.09913192078470735302264671418015495873987674713134765625000000000000000000000000
0.3086386756624810989713125763955758884549140930175781250000000000
0.09871205569793220502283759287820430472493171691894531250000000000000000000000000
0.4073507313604133317497257849026937037706375122070312500000000000
0.09828798032688000618772861116667627356946468353271484375000000000000000000000000
0.5056387116872933518152422038838267326354980468750000000000000000
0.0978596635449100332726501960678

4.7091273930473018793918527080677449703216552734375000000000000000
0.07137413157038387245822974591646925546228885650634765625000000000000000000000000
4.7805015246176854049053872586227953433990478515625000000000000000
0.07072688654540928987479730949416989460587501525878906250000000000000000000000000
4.8512284111630945560023064899723976850509643554687500000000000000
0.07007625920566276644940728601795854046940803527832031250000000000000000000000000
4.9213046703687570726515332353301346302032470703125000000000000000
0.06942235214966999135022263089922489598393440246582031250000000000000000000000000
4.9907270225184268142015753255691379308700561523437500000000000000
0.06876527246422575734285942417045589536428451538085937500000000000000000000000000
5.0594922949826521829663761309348046779632568359375000000000000000
0.06810513173289750588068613978975918143987655639648437500000000000000000000000000
5.1275974267155497443582135019823908805847167968750000000000000000
0.0674420460358327866767780278678

7.6335823807854037426068316563032567501068115234375000000000000000
0.03470657943370632214641346990902093239128589630126953125000000000000000000000000
7.6682889602191099953643060871399939060211181640625000000000000000
0.03411068321267347186465102026886597741395235061645507812500000000000000000000000
7.7023996434317831827343070472124963998794555664062500000000000000
0.03352052790667285619985449329760740511119365692138671875000000000000000000000000
7.7359201713384564413900079671293497085571289062500000000000000000
0.03293623253051022187198881852054910268634557723999023437500000000000000000000000
7.7688564038689662538672564551234245300292968750000000000000000000
0.03235791016500432631941563954569573979824781417846679687500000000000000000000000
7.8012143140339702540586586110293865203857421875000000000000000000
0.03178566785868228855571615554254094604402780532836914062500000000000000000000000
7.8329999818926525634310564782936125993728637695312500000000000000
0.0312196065454046871312510091911

8.8281395066274175320586437010206282138824462890625000000000000000
0.01161230762397383667328121248374372953549027442932128906250000000000000000000000
8.8397518142513913375069023459218442440032958984375000000000000000
0.01136497756020569786794549571595780435018241405487060546875000000000000000000000
8.8511167918115969399650566629134118556976318359375000000000000000
0.01112262358689789648602541660693532321602106094360351562500000000000000000000000
8.8622394153984949127789150224998593330383300781250000000000000000
0.01088516395589094604146040978775999974459409713745117187500000000000000000000000
8.8731245793543855882035131799057126045227050781250000000000000000
0.01065251708571711593276365448446085792966187000274658203125000000000000000000000
8.8837770964401023832124337786808609962463378906250000000000000000
0.01042460164225536556337203819566639140248298645019531250000000000000000000000000
8.8942016980823570548864154261536896228790283203125000000000000000
0.0102013366145202767631960938388

9.2042907463723260264032433042302727699279785156250000000000000000
0.00347751605252485116240301898926645662868395447731018066406250000000000000000000
9.2077682624248513576503682998009026050567626953125000000000000000
0.00340132593820575935758543373310658353148028254508972167968750000000000000000000
9.2111695883630577696976615698076784610748291015625000000000000000
0.00332678674631080219942225184581729990895837545394897460937500000000000000000000
9.2144963751093680315307210548780858516693115234375000000000000000
0.00325386321629713782047077508252641564467921853065490722656250000000000000000000
9.2177502383256655349441643920727074146270751953125000000000000000
0.00318252081883686798249866178878164646448567509651184082031250000000000000000000
9.2209327591445031657713116146624088287353515625000000000000000000
0.00311272574232965455742760951807213132269680500030517578125000000000000000000000
9.2240454848868331794164987513795495033264160156250000000000000000
0.0030444448795136380171710843711

...

9.3858571202657241627775874803774058818817138671875000000000000000
0.00456499944365242223132117516115613398142158985137939453125000000000000000000000
9.3904221197093757922402801341377198696136474609375000000000000000
0.00540831718448869271098633504379904479719698429107666015625000000000000000000000
9.3958304368938652118004029034636914730072021484375000000000000000
0.00640375649836285015109593743432014889549463987350463867187500000000000000000000
9.4022341933922284340496844379231333732604980468750000000000000000
0.00757894030529744112956169743711143382824957370758056640625000000000000000000000
9.4098131336975256999721750617027282714843750000000000000000000000
0.00896628081659234417444270803798644919879734516143798828125000000000000000000000
9.4187794145141179313895918312482535839080810546875000000000000000
0.01060418818674377719468715497441735351458191871643066406250000000000000000000000
9.4293836027008612887811977998353540897369384765625000000000000000
0.0125387130034190705241137209213

10.5775289299592181180287298047915101051330566406250000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.5875289299592179048659090767614543437957763671875000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.5975289299592176917030883487313985824584960937500000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.6075289299592174785402676207013428211212158203125000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.6175289299592172653774468926712870597839355468750000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.6275289299592170522146261646412312984466552734375000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.6375289299592168390518054366111755371093750000000000000000000000
0.010000000000000000208166

11.1375289299592061809107690351083874702453613281250000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1475289299592059677479483070783317089080810546875000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1575289299592057545851275790482759475708007812500000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1675289299592055414223068510182201862335205078125000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1775289299592053282594861229881644248962402343750000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1875289299592051150966653949581086635589599609375000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.1975289299592049019338446669280529022216796875000000000000000000
0.010000000000000000208166

11.6975289299591942437928082654252648353576660156250000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7075289299591940306299875373952090740203857421875000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7175289299591938174671668093651533126831054687500000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7275289299591936043043460813350975513458251953125000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7375289299591933911415253533050417900085449218750000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7475289299591931779787046252749860286712646484375000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
11.7575289299591929648158838972449302673339843750000000000000000000
0.010000000000000000208166

12.2575289299591823066748474957421422004699707031250000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.2675289299591820935120267677120864391326904296875000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.2775289299591818803492060396820306777954101562500000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.2875289299591816671863853116519749164581298828125000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.2975289299591814540235645836219191551208496093750000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.3075289299591812408607438555918633937835693359375000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
12.3175289299591810276979231275618076324462890625000000000000000000
0.010000000000000000208166

In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

7462


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

986.9288859425887 seconds
16.448814765709812 minutes
0.27414691276183023 hours
74620000 points
75608.28451052224 points per second
Took 11.33 less minutes
